# lncRNA pipeline

In [ ]:
Files needed..

In [2]:
blastout="analyses/Myt-sxt-trinity2_blastx_sp.out"
fasta="../data/Mytilus-SXT.trimmed.trinity.fa"

In [3]:
#all with evalue <1e-5
!wc -l {blastout}
!head -3 {blastout}


   29486 analyses/Myt-sxt-trinity2_blastx_sp.out
TR4_c0_g1_i1	sp|Q8R151|ZNFX1_MOUSE	53.25	77	35	1	2	229	1462	1538	1e-16	78.2
TR7_c0_g1_i1	sp|Q8VHI5|VITRN_MOUSE	40.32	62	37	0	170	355	60	121	4e-07	52.0
TR11_c0_g1_i1	sp|Q16674|MIA_HUMAN	39.60	101	47	3	36	296	10	110	3e-11	62.0


## sort blast table for joining

In [5]:
!sort {blastout} \
> {blastout}.sorted
!head -2 {blastout}.sorted

TR10000_c0_g2_i1	sp|Q5RCP3|NAA38_PONAB	76.04	96	23	0	30	317	1	96	1e-45	  159
TR10004_c0_g1_i1	sp|Q6P2X9|MOT12_XENTR	42.57	101	58	0	367	65	95	195	2e-15	75.5


## create tab file from transcriptome

In [6]:
#clean up fasta ID to just be contig id
#MIGHT not be needed
!cut -d' ' -f1 {fasta} \
> {fasta}IDonly
!head {fasta}IDonly

>TR1_c0_g1_i1
GGCTTATCTGTCATAGTCCGCGGAAATACTTTATTTAGAATTTCAACCTGAAAACATGCG
AAACAATACTATACATGCATTATGTATCAAGTGCATAAAACAGCAGTGGGGAAACAACAA
AAGAAAAATGCATTAAACAGAATAAAAATAAATTGCATAAACAAAACTGTATACTTACTT
AAACACTAAAATAATTGAAAAAAGATAATTGAACCTGCATTGGTTGCAACGTAAACATGT
ACTTACCAG
>TR2_c0_g1_i1
CAATAAGTCCTTACGTAAAAATTAAATTGATGCCGCGTTCCAAAATGGGACATAAAAGGC
TTCTCTTTTAGTGAATAATTTGTTGTTACAGCATTAAAAAAAAAATCACTCACTTATAAC
TATATATTTATTAAAGTATATTTCTAAGGAAAATGTTCTATCATAGCAAAATATCAGAAA


In [7]:
#create tab-delimited file
!perl -e '$count=0; $len=0; while(<>) {s/\r?\n//; s/\t/ /g; if (s/^>//) { if ($. != 1) {print "\n"} s/ |$/\t/; $count++; $_ .= "\t";} else {s/ //g; $len += length($_)} print $_;} print "\n"; warn "\nConverted $count FASTA records in $. lines to tabular format\nTotal sequence length: $len\n\n";' \
{fasta}IDonly  \
| sort > {fasta}IDonly.tab

Can't open {fasta}IDonly: No such file or directory at -e line 1.

Converted 0 FASTA records in 0 lines to tabular format
Total sequence length: 0



In [24]:
!head analyses/Mytilus-SXT.trimmed.trinityIDonly.TAB

TR10000_c0_g1_i1		CTCATCTTGATTTATCTTAATCTTTGAAAATGGCTTCAGCATTGGAATCGTATGTGAATCGTACAGTTTCTATAGTCACAGCAGATGGAAGAATAATTGTGGGAACATTGAAAGGATTTGATCAAACAATCAACTTAATTCTAGACGAGAGCCATGAGAGGGTATATAGTGCTACACAAGGAGTAGAACAAGTCATTTTGGGACTATACATCATCAGGGGAGATAATGTAGCTTTGATAGGAGAGATAGATGATGAAGCTGACCAGAGATTAGATTTTAATGACATTAAGGCTGAACCTTTGAATTCAGTTTTACATTGATGTAAAGGATAAAAAGACATGTTATGGCTTGGAAATAAATCTCAGCTTAGTTGAATTACAGGAATTTTAGGACTTACAACCAAAGAGAATGTACATTTAATTACATAAATAAATGCAAGCTGATCAAGAGAGAGAGGGAGATTTTCAATACTTGTGTATGGTTTTGTGCTTAATTAACATTTTTAAAGTGTTAAAAGTGTCATAATATATTTTGAAGAATGCCTCCCTAACCTGTAAAGGAAGATTGATTATTTATAGCAATTACTTAAGGAAAAATAATAAATATTCAACATGTTCAAACTAAATTCATTTTTATTAAGTTAAGATAATTGAATGAGATGTTTGATTGTTTCTGTTGGTTCACAATTCCATGTCTAGGTTCAACTGACCTAGACATGGAATTGTGAA
TR10000_c0_g2_i1		CTCATCTTGATTTATCTTAATCTTTGAAAATGGCTTCAGCATTGGAATCGTATGTGAATCGTACAGTTTCTATAGTCACAGCAGATGGAAGAATAATTGTGGGAACATTGAAAGGATTTGATCAAACAATCAACTTAATTCTAGACGAGAGCCATGAGAGGGTATATAGTGCTACACAAGGAGTAGAACAAGTCATTTTGGGACTATACATCATCAGGGGAGATAATGTAGCTTT

In [25]:
!wc -l analyses/Mytilus-SXT.trimmed.trinityIDonly.TAB

  184834 analyses/Mytilus-SXT.trimmed.trinityIDonly.TAB


In [29]:
!join -v 1 analyses/Mytilus-SXT.trimmed.trinityIDonly.TAB \
analyses/Myt-sxt-trinity2_blastx_sp.sorted > analyses/Mytilus-SXT.trimmed.trinityIDonly.noblast

In [7]:
!head -3 analyses/Mytilus-SXT.trimmed.trinityIDonly.noblast
!wc -l analyses/Mytilus-SXT.trimmed.trinityIDonly.noblast

TR10000_c0_g1_i1 CTCATCTTGATTTATCTTAATCTTTGAAAATGGCTTCAGCATTGGAATCGTATGTGAATCGTACAGTTTCTATAGTCACAGCAGATGGAAGAATAATTGTGGGAACATTGAAAGGATTTGATCAAACAATCAACTTAATTCTAGACGAGAGCCATGAGAGGGTATATAGTGCTACACAAGGAGTAGAACAAGTCATTTTGGGACTATACATCATCAGGGGAGATAATGTAGCTTTGATAGGAGAGATAGATGATGAAGCTGACCAGAGATTAGATTTTAATGACATTAAGGCTGAACCTTTGAATTCAGTTTTACATTGATGTAAAGGATAAAAAGACATGTTATGGCTTGGAAATAAATCTCAGCTTAGTTGAATTACAGGAATTTTAGGACTTACAACCAAAGAGAATGTACATTTAATTACATAAATAAATGCAAGCTGATCAAGAGAGAGAGGGAGATTTTCAATACTTGTGTATGGTTTTGTGCTTAATTAACATTTTTAAAGTGTTAAAAGTGTCATAATATATTTTGAAGAATGCCTCCCTAACCTGTAAAGGAAGATTGATTATTTATAGCAATTACTTAAGGAAAAATAATAAATATTCAACATGTTCAAACTAAATTCATTTTTATTAAGTTAAGATAATTGAATGAGATGTTTGATTGTTTCTGTTGGTTCACAATTCCATGTCTAGGTTCAACTGACCTAGACATGGAATTGTGAA
TR10001_c0_g1_i1 ATAATAAAATCAAATTTGGTTATCAAACTTATTTATCCTAATATCAATACTAAATTTAATTACTATTGCTTAGAGGTGTGGAATAAATTGTCAAGCTCGATATCGACAAACAACATCCTAAACCACAAGAAATGCGCAATATGTATTAAAATACCAGACGACATAGTATAGGGATTATCTAATGCATGCATCATGATTTAATTAGAACGCATTAATTATTTCGGTTAACCAGATGTC

In [8]:
!awk '{print ">"$1"\n"$2}' \
analyses/Mytilus-SXT.trimmed.trinityIDonly.noblast \
> analyses/Mytilus-SXT.trimmed.trinityNOHIT.fa
!head analyses/Mytilus-SXT.trimmed.trinityNOHIT.fa


>TR10000_c0_g1_i1
CTCATCTTGATTTATCTTAATCTTTGAAAATGGCTTCAGCATTGGAATCGTATGTGAATCGTACAGTTTCTATAGTCACAGCAGATGGAAGAATAATTGTGGGAACATTGAAAGGATTTGATCAAACAATCAACTTAATTCTAGACGAGAGCCATGAGAGGGTATATAGTGCTACACAAGGAGTAGAACAAGTCATTTTGGGACTATACATCATCAGGGGAGATAATGTAGCTTTGATAGGAGAGATAGATGATGAAGCTGACCAGAGATTAGATTTTAATGACATTAAGGCTGAACCTTTGAATTCAGTTTTACATTGATGTAAAGGATAAAAAGACATGTTATGGCTTGGAAATAAATCTCAGCTTAGTTGAATTACAGGAATTTTAGGACTTACAACCAAAGAGAATGTACATTTAATTACATAAATAAATGCAAGCTGATCAAGAGAGAGAGGGAGATTTTCAATACTTGTGTATGGTTTTGTGCTTAATTAACATTTTTAAAGTGTTAAAAGTGTCATAATATATTTTGAAGAATGCCTCCCTAACCTGTAAAGGAAGATTGATTATTTATAGCAATTACTTAAGGAAAAATAATAAATATTCAACATGTTCAAACTAAATTCATTTTTATTAAGTTAAGATAATTGAATGAGATGTTTGATTGTTTCTGTTGGTTCACAATTCCATGTCTAGGTTCAACTGACCTAGACATGGAATTGTGAA
>TR10001_c0_g1_i1
ATAATAAAATCAAATTTGGTTATCAAACTTATTTATCCTAATATCAATACTAAATTTAATTACTATTGCTTAGAGGTGTGGAATAAATTGTCAAGCTCGATATCGACAAACAACATCCTAAACCACAAGAAATGCGCAATATGTATTAAAATACCAGACGACATAGTATAGGGATTATCTAATGCATGCATCATGATTTAATTAGAACGCATTAATTATTTCGGTTAACCAGATG

In [9]:
!fgrep -c ">" analyses/Mytilus-SXT.trimmed.trinityNOHIT.fa

155348


# Gigas ncRNA Comparison

In [10]:
blastloc="/Applications/transrate-1.0.1-osx/lib/app/ruby/2.2.0/bin/"

In [12]:
!{blastloc}makeblastdb \
-dbtype nucl \
-in ../data/Mytilus-SXT.trimmed.trinity.v3.fa \
-out analyses/Mytilus-SXT.trimmed.trinity.v3



Building a new DB, current time: 12/11/2015 13:00:37
New DB name:   analyses/Mytilus-SXT.trimmed.trinity.v3
New DB title:  ../data/Mytilus-SXT.trimmed.trinity.v3.fa
Sequence type: Nucleotide
Keep Linkouts: T
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 184166 sequences in 27.9305 seconds.


In [15]:
!{blastloc}blastn \
-task blastn \
-query ../data/Crassostrea_gigas.GCA_000297895.1.29.ncrna.fa \
-db analyses/Mytilus-SXT.trimmed.trinity.v3 \
-evalue 1e-5 \
-outfmt 6 \
-num_threads 2 \
-out analyses/Cgigas-ncrna-blastn-Mytv3.out \
2> analyses/err_Cgigas-ncrna-blastn-Mytv3